In [1]:
%pip install gymnasium stable-baselines3 torch
%pip install stable-baselines3[extra]
%pip install sb3-contrib
%pip install torch torchvision
%pip install numpy protobuf
%pip install onnx
%pip install pettingzoo[all] ray[rllib] supersuit

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 852.5/852.5 kB 4.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.4/68.4 MB 3.6 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 139.1 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 552.0/552.0 kB 636.2 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 3.4 MB/s eta 0:00:0000:0100:010m
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.0/269.0 kB 262.6 kB/s e

In [3]:

%pip install pettingzoo[all] ray[rllib] supersuit

  Using cached pettingzoo-1.25.0-py3-none-any.whl (852 kB)
  Using cached ray-2.45.0-cp311-cp311-manylinux2014_x86_64.whl (68.4 MB)
  Using cached supersuit-3.10.0-py3-none-any.whl (50 kB)
  Using cached multi-agent-ale-py-0.1.11.tar.gz (551 kB)
  Preparing metadata (setup.py) ... done
  Using cached chess-1.11.2-py3-none-any.whl
  Using cached rlcard-1.2.0-py3-none-any.whl
  Using cached Shimmy-2.0.0-py3-none-any.whl (30 kB)
  Using cached pymunk-6.11.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
  Using cached box2d-py-2.3.8.tar.gz (374 kB)
  Preparing metadata (setup.py) ... done
  Using cached tensorboardX-2.6.2.2-py2.py3-none-any.whl (101 kB)
  Using cached dm_tree-0.1.9-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (152 kB)
  Using cached gymnasium-1.0.0-py3-none-any.whl (958 kB)
  Using cached ormsgpack-1.7.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (220 kB)
  Using cached tinyscaler-1.2.8-cp311-cp311-manylinux_2_17_x86_64.m

# 強化学習モデルの学習 (main.py)

このセルでは、DQNアルゴリズムを用いて、`CartPole-v1`環境でモデルを学習させます。

In [1]:
import gymnasium as gym
import torch
import ray
from ray import tune
from ray.rllib.env import PettingZooEnv
from cat_toy_env import CatToyEnv
from pettingzoo.utils import parallel_to_aec
from ray.tune.registry import register_env
from ray.rllib.algorithms.ppo import PPOConfig
from pettingzoo.utils.conversions import parallel_wrapper_fn
import supersuit
from cat_toy_env import CatToyEnv

# 256MB = 256 * 1024 * 1024
ray.shutdown()
ray.init(object_store_memory=256 * 1024 * 1024)

2025-05-04 20:28:49,093	INFO worker.py:1888 -- Started a local Ray instance.


Python version:,3.11.3
Ray version:,2.45.0


In [2]:
import json
import os

# 設定ファイルの読み込み
with open('../cat-game/config/common.json', 'r') as f:
    common_config = json.load(f)

In [3]:
def env_creator(config=None):
    return CatToyEnv(config=common_config)

register_env("cat_toy", env_creator)

# エージェントごとのポリシー設定
policies = {
    "cat_policy": (None, env_creator().observation_space, env_creator().action_space, {}),
    "toy_policy": (None, env_creator().observation_space, env_creator().action_space, {}),
}

# エージェントIDとポリシーのマッピング
def policy_mapping_fn(agent_id, episode, worker, **kwargs):
    return f"{agent_id}_policy"

In [ ]:
# PPOの設定
config = (
    PPOConfig()
    .environment(env=PettingZooEnv(lambda config: CatToyEnv(config=config)), 
                 env_config = common_config)
    .multi_agent(policies=policies, policy_mapping_fn=policy_mapping_fn)
    .framework("torch")
    .env_runners(num_env_runners=0)  # 並列なし
    .env_runners(num_cpus_per_env_runner=0.5)  # ワーカーごとのCPUも削減
)

# 学習の実行
tune.run("PPO", config=config.to_dict(), stop={"episodes_total": 1})#10000})

AttributeError: 'function' object has no attribute 'reset'

In [ ]:
# 入力の2つのTensorを結合
toy = torch.randn(1, 2)
cat = torch.randn(1, 2)
concat_input = torch.cat([toy, cat], dim=1)  # shape: (1, 4)

# エクスポート対象モデル（例: policyネットワーク）
policy_net = loaded_model.policy.q_net

# ONNXエクスポート
torch.onnx.export(
    policy_net,
    concat_input,  # ← dictではなく単一Tensor
    "cat_dqn_policy.onnx",
    export_params=True,
    opset_version=11,
    input_names=["obs"],
    output_names=["q_values"],
    dynamic_axes={
        "obs": {0: "batch_size"},
        "q_values": {0: "batch_size"}
    }
)


In [ ]:
# 環境のクローズ
env_learning.close()
env_eval.close()

# 学習済みモデルの使用 (play.py)

このセルでは、学習済みのモデルをロードし、`CartPole-v1`環境でエージェントがどのように行動するかを観察します。

In [ ]:
import gymnasium as gym
from stable_baselines3 import DQN
import time
from cat_toy_env import CatToyEnv

In [ ]:
env_kwargs=dict(render_mode="human", max_steps=1000)

# 環境の作成
env = CatToyEnv(**env_kwargs)

# モデルのロード
model_playing = DQN.load("cat_dqn")

In [ ]:
# エピソードの実行
obs, info = env.reset()
done = False
while not done:
    action, _states = model_playing.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)
    print("観測:", obs)
    done = terminated or truncated
    env.render()  # 環境の描画
    #time.sleep(0.001) # 0.01秒待機

In [ ]:
# 環境のクローズ
env.close()